In [1]:
"""
This class implements Non-linear Matrix Factorization with Gaussian Processes (NLMFGP), described in:
    Lawrence, Neil D., and Raquel Urtasun.
    "Non-linear matrix factorization with Gaussian processes."
    Proceedings of the 26th Annual International Conference on Machine Learning. ACM, 2009.
"""

import sys
import os
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import linear_kernel, rbf_kernel
import time
from sklearn.model_selection import train_test_split


# Add parent directory to python path
#PACKAGE_PARENT = '..'
#SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser(__file__))))
#sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
sys.path.append(r'E:\Developer\Python\Myworkshop\Python_Study\机器学习\Matrix Factorization\nonlin_gp_mf')
sys.path.append(r'E:\Developer\Python\Myworkshop\Python_Study\机器学习\Matrix Factorization\data_fetching')
from data_set import DataSet


class GpMf():
    def __init__(self, latent_dim, nb_data):
        self.latent_dim = latent_dim
        self.nb_data = nb_data
        self.X = np.random.normal(0, 1e-3, (nb_data, latent_dim))
        self.lin_variance = 1.0
        self.bias_variance = 0.11
        self.white_variance = 5.0
        self.y = None
        self.rated_items = None

    def log_likelihood(self):
        """return the log likelihood of the model"""
        Cj_invy, logDetC = self.invert_covariance()
        yj = np.asmatrix(self.y).T
        Nj = len(self.rated_items)
        likelihood = - 0.5 * (Nj * np.log(2 * math.pi) + logDetC + yj.T.dot(Cj_invy))
        return float(likelihood)

    def invert_covariance(self, gradient=False, nonlinear =False, kernel=linear_kernel):
        q = self.latent_dim
        Nj = len(self.rated_items)
        Xj = np.asmatrix(self.X[self.rated_items, :])
        #print("self.rated_items.shape:",self.rated_items.shape)
        #print("invert_covariance----self.X[self.rated_items, :]:",self.X[self.rated_items, :])
        yj = np.asmatrix(self.y).T
        s_n = self.white_variance
        s_w = self.lin_variance
        s_b = self.bias_variance
        sigNoise = s_w / s_n

        if Nj > q and not nonlinear: # we use the matrix inversion lemma
            XTX = Xj.T * Xj
            B = np.eye(q) + sigNoise * XTX
            Binv = np.linalg.pinv(B)
            _, logdetB = np.linalg.slogdet(B)
            if gradient:
                AinvX = (Xj - sigNoise * Xj * (Binv * XTX)) / s_n
                AinvTr = (Nj - sigNoise * (np.multiply(Xj * Binv, Xj)).sum()) / s_n
            Ainvy = (yj - sigNoise * Xj * (Binv * (Xj.T * yj))) / s_n
            sumAinv = (np.ones((Nj, 1)) - sigNoise * Xj * (Binv * Xj.sum(axis=0).T)) / s_n  # this is Nx1
            sumAinvSum = sumAinv.sum()
            denom = 1 + s_b * sumAinvSum
            fact = s_b / denom
            if gradient:
                CinvX = AinvX - fact * sumAinv * (sumAinv.T * Xj)
                CinvSum = sumAinv - fact * sumAinv * sumAinvSum
                CinvTr = AinvTr - fact * sumAinv.T * sumAinv

            Cinvy = Ainvy - fact * sumAinv * float(sumAinv.T * yj)
            if not gradient:
                logdetA = Nj * np.log(s_n) + logdetB
                logdetC = logdetA + np.log(denom)

        else :
            C = s_w * kernel(Xj, Xj)
            C = C + s_b + s_n * np.eye(Nj)
            Cinv = np.linalg.pinv(C)
            Cinvy = Cinv * yj
            if gradient:
                CinvX = Cinv * Xj
                CinvTr = np.trace(Cinv)
                CinvSum = Cinv.sum(axis=1)
            else:
                _, logdetC = np.linalg.slogdet(C)

        if gradient:
            return Cinvy, CinvSum, CinvX, CinvTr
        else:
            return Cinvy, logdetC

    def log_likelihood_grad(self, ):
        """Computes the gradient of the log likelihood"""
        s_w = self.lin_variance
        s_b = self.bias_variance
        s_n = self.white_variance

        yj = np.asmatrix(self.y).T
        Xj = np.asmatrix(self.X[self.rated_items, :])

        Cinvy, CinvSum, CinvX, CinvTr = self.invert_covariance(gradient=True)
        covGradX = 0.5 * (Cinvy * (Cinvy.T * Xj) - CinvX)
        gX = s_w * 2.0 * covGradX
        gsigma_w = np.multiply(covGradX, Xj).sum()
        CinvySum = Cinvy.sum()
        CinvSumSum = CinvSum.sum()
        gsigma_b = 0.5 * (CinvySum * CinvySum - CinvSumSum)
        gsigma_n = 0.5 * (Cinvy.T * Cinvy - CinvTr)
        return gX, float(gsigma_w), float(gsigma_b), float(gsigma_n)

    def objective(self):
        return -self.log_likelihood()


In [2]:
def fit(dataset, model, nb_iter=10, seed=42, momentum=0.9):
    data = dataset.get_df()
    param_init = np.zeros((1, 3))
    X_init = np.zeros(model.X.shape)
    for iter in range(nb_iter):
        print("iteration", iter)
        tic = time.time()
        np.random.seed(seed=seed)
        state = np.random.get_state()
        users = np.random.permutation(dataset.get_users())
        for user in users:
            #print("begin user", user,  "=========================")
            toc = time.time()
            lr = 1e-4
            y = dataset.get_ratings_user(user)
            #print('dataset.get_ratings_user(user):', y)
            #获取每个人的评分，打印结果如下：
            #dataset.get_ratings_user(user): [3 5 5 5 5 5 3 4 5 5 5 4 5 4 5 4 5 3 4 4 5 5 4 1 5 5 5 5 4 4 5 5 5 3 5 3 5
            #                                 3 5 2 5 5 5 5 5 4 4 4 5 5 2 1 1 5 2 5 4 3 3 4 5 5]
            #dataset.get_ratings_user(user): [3 2 4 4 4 4 3 4 4 3 4 5 4 3 2 1 3 3 5 1 3 2]
            #dataset.get_ratings_user(user): [5 3 5 4 3 5 5 4 2 4 5 4 5 4 5 4 3 5 3 4]
            #获取每个人的评分的（影片ID-1）
            rated_items = dataset.get_items_user(user) - 1

            model.y = y
            model.rated_items = rated_items
            #print("model.rated_items:",model.rated_items)
            grad_X, grad_w, grad_b, grad_n = model.log_likelihood_grad()
            gradient_param = np.array([grad_w * model.lin_variance,
                               grad_b * model.bias_variance,
                               grad_n * model.white_variance])
            param = np.log(np.array([model.lin_variance,
                                     model.bias_variance,
                                     model.white_variance]))
            # update X
            X = X_init[rated_items, :]
            ar = lr * 10
            X = X * momentum + grad_X * ar
            X_init[rated_items, :] = X
            model.X[rated_items, :] = model.X[rated_items, :] + X

            # update variances
            param_init = param_init * momentum + gradient_param * lr
            param = param + param_init
            model.lin_variance = math.exp(param[0, 0])
            model.bias_variance = math.exp(param[0, 1])
            model.white_variance = math.exp(param[0, 2])
            #print("end user", user, "=========================")

        print("end iteration", iter,  "=========================")
        print("duration iteration", time.time() - tic)
    return model


In [3]:
def predict(user, test_items, model, dataset):
    y = dataset.get_ratings_user(user)
    rated_items = dataset.get_items_user(user) - 1
    model.rated_items = rated_items
    model.y = y
    X_test = np.asmatrix(model.X[test_items, :])
    X = np.asmatrix(model.X[model.rated_items, :])
#     print("test_items:",test_items,test_items.shape)
    print("X:",X.shape)
    print("X_test:", X_test.shape)
    print("model.X[test_items, :]",model.X[test_items, :].shape)
    print("model.X[model.rated_items, :]",model.X[model.rated_items, :].shape)
    
    Cinvy, CinvSum, CinvX, CinvTr = model.invert_covariance(gradient=True)
    mean = model.lin_variance* X_test*(X.T*Cinvy) + Cinvy.sum() * model.bias_variance
    return mean


In [4]:
def perf_weak(dataset, base_dim=9):
    print('Fetch data set...')
    if dataset.dataset == "movielens":
        norm_coeff = 1.6
        print("norm_coeff:1.6------------")
    else :
        norm_coeff = 6.67
    print('Data set fetched')
    print("Dataset desctiption", dataset.get_description())
    model_init = GpMf(latent_dim=base_dim, nb_data=dataset.item_index_range)
    print('Fit the model...')
    model = fit(dataset=dataset, model=model_init)
    print('Model fitted')
    predictions = []
    true_ratings = []
    test_users = dataset.get_users_test()
    nb_users_test = len(test_users)
    print("nb_users", nb_users_test)
    count = 0
    for user in test_users:
        prediction = predict(user, dataset.get_item_test(user) - 1, model, dataset)
        print( "prediction: ",prediction )

        if prediction > dataset.high_rating:
            prediction = dataset.high_rating
        if prediction < dataset.low_rating:
            prediction = dataset.low_rating
        predictions.append(prediction)
        rating = dataset.get_rating_test(user)
        true_ratings.append(rating)
        count += 1
        print(count, "over ", nb_users_test, "users")
    predictions = np.asarray(predictions)
    true_ratings = np.asarray(true_ratings)
    rmse = np.linalg.norm(predictions - true_ratings) / np.sqrt(nb_users_test)
    nmae = np.sum(np.abs(true_ratings - predictions)) * 1. / (len(predictions) * norm_coeff)
    print("rmse", rmse)
    print("nmae", nmae)
    return float(rmse), float(nmae)


In [5]:
# ========================== DEMO ======================================================================================
def plot_errors_vs_latent_dims():
    #base_dims = range(5, 30)  #latent feature = 9,最好
    base_dims = 9
    rmse_res = []
        #0.9192805345815771, 0.9191762806556765, 0.9298582192495648, 0.9264887573748214, 0.942721413670847,
        #0.9464276649204383, 0.9619938797137044, 0.959524838407498, 0.9637647705458857, 0.9654112679112156,
        #0.964014562524729, 0.9828162923399141, 0.9791815989138641, 0.9755912980143179, 0.9909916313775403]
    nmae_res = []
        #0.44971496453721255, 0.44916990674349777, 0.4520802986328138, 0.45121143493891314, 0.45841600020196077,
        #0.4600831341641973, 0.46819195277564857, 0.4653186575931832, 0.4675059307434209, 0.46521752440367997,
        #0.4693989965956004, 0.4749311584164483, 0.47302261002602103, 0.47072878663284334, 0.4775780782441432]

    if not len(rmse_res):
        dataset_movielens_s = DataSet(dataset="movielens", size="S")
        for dim in base_dims:
            (rmse, nmae) = perf_weak(dataset=dataset_movielens_s, base_dim=dim)
            rmse_res.append(rmse)
            nmae_res.append(nmae)

        print(rmse_res)
        print(nmae_res)

    plt.figure()
    plt.plot(base_dims, rmse_res, marker='.')
    plt.grid()
    plt.xlabel('Number of latent dimensions ($r$)')
    plt.ylabel('RMSE (MovieLens 100k)')
    plt.show()

    plt.figure()
    plt.plot(base_dims, nmae_res, marker='*')
    plt.xlabel('Number of latent dimensions ($r$)')
    plt.ylabel('NMAE (MovieLens 100k)')
    plt.grid()
    plt.show()


In [6]:
print('START')
dataset=DataSet(dataset="movielens", size="S")
base_dim = 9
#----------------------------------------------------
print('Fetch data set...')
if dataset.dataset == "movielens":
    norm_coeff = 1.6
    print("norm_coeff:1.6------------")
else :
    norm_coeff = 6.67
print('Data set fetched')
print("Dataset desctiption", dataset.get_description())
model_init = GpMf(latent_dim=base_dim, nb_data=dataset.item_index_range)
print('Fit the model...')
model = fit(dataset=dataset, model=model_init)
print('Model fitted')


START
Fetch data set...
norm_coeff:1.6------------
Data set fetched
Dataset desctiption {'Number of users': 943, 'Number of items': 1682, 'Lowest user': 1, 'Highest user': 943, 'Density': 0.06304669364224531, 'Mean of ratings': 3.52986, 'Standard deviation of ratings': 1.125667970762062}
Fit the model...
iteration 0
end iteration 0 =========================
duration iteration 19.964471578598022
iteration 1
end iteration 1 =========================
duration iteration 17.857333660125732
iteration 2
end iteration 2 =========================
duration iteration 17.02627444267273
iteration 3
end iteration 3 =========================
duration iteration 17.659451484680176
iteration 4
end iteration 4 =========================
duration iteration 17.06828761100769
iteration 5
end iteration 5 =========================
duration iteration 17.358850955963135
iteration 6
end iteration 6 =========================
duration iteration 18.074206829071045
iteration 7
end iteration 7 ========================

In [7]:
predictions = []
true_ratings = []
test_users = dataset.get_users_test()
nb_users_test = len(test_users)
print("nb_users", nb_users_test)
count = 0
for user in test_users:
    prediction = predict(user, dataset.get_item_test(user) - 1, model, dataset)
    print("dataset.get_item_test(user) - 1:",dataset.get_item_test(user) - 1)
    print( "prediction: ",prediction )

    if prediction > dataset.high_rating:
        prediction = dataset.high_rating
    if prediction < dataset.low_rating:
        prediction = dataset.low_rating
    predictions.append(prediction)
    rating = dataset.get_rating_test(user)
    true_ratings.append(rating)
    count += 1
    print(count, "over ", nb_users_test, "users")
predictions = np.asarray(predictions)
true_ratings = np.asarray(true_ratings)
rmse = np.linalg.norm(predictions - true_ratings) / np.sqrt(nb_users_test)
nmae = np.sum(np.abs(true_ratings - predictions)) * 1. / (len(predictions) * norm_coeff)
print("rmse", rmse)
print("nmae", nmae)

print('END')

nb_users 943
无int操作 get_item_test: 35409    271
Name: item_id, dtype: int64
X: (271, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (271, 9)
无int操作 get_item_test: 35409    271
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 270
prediction:  [[2.95484911]]
无float操作 get_item_test: 35409    2
Name: rating, dtype: int64
1 over  943 users
无int操作 get_item_test: 5324    290
Name: item_id, dtype: int64
X: (61, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (61, 9)
无int操作 get_item_test: 5324    290
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 289
prediction:  [[3.17697115]]
无float操作 get_item_test: 5324    3
Name: rating, dtype: int64
2 over  943 users
无int操作 get_item_test: 24716    260
Name: item_id, dtype: int64
X: (53, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (53, 9)
无int操作 get_item_test: 24716    260
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 259
predict

model.X[model.rated_items, :] (67, 9)
无int操作 get_item_test: 15341    216
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 215
prediction:  [[4.4557741]]
无float操作 get_item_test: 15341    4
Name: rating, dtype: int64
24 over  943 users
无int操作 get_item_test: 53    181
Name: item_id, dtype: int64
X: (77, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (77, 9)
无int操作 get_item_test: 53    181
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 180
prediction:  [[4.2677532]]
无float操作 get_item_test: 53    5
Name: rating, dtype: int64
25 over  943 users
无int操作 get_item_test: 91015    936
Name: item_id, dtype: int64
X: (106, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (106, 9)
无int操作 get_item_test: 91015    936
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 935
prediction:  [[3.52572314]]
无float操作 get_item_test: 91015    4
Name: rating, dtype: int64
26 over  943 users
无int操作 get_item_test: 19167    325

无int操作 get_item_test: 47255    136
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 135
prediction:  [[2.9402215]]
无float操作 get_item_test: 47255    4
Name: rating, dtype: int64
51 over  943 users
无int操作 get_item_test: 33118    111
Name: item_id, dtype: int64
X: (55, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (55, 9)
无int操作 get_item_test: 33118    111
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 110
prediction:  [[4.36511595]]
无float操作 get_item_test: 33118    4
Name: rating, dtype: int64
52 over  943 users
无int操作 get_item_test: 5866    100
Name: item_id, dtype: int64
X: (27, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (27, 9)
无int操作 get_item_test: 5866    100
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 99
prediction:  [[4.59464699]]
无float操作 get_item_test: 5866    5
Name: rating, dtype: int64
53 over  943 users
无int操作 get_item_test: 33674    288
Name: item_id, dtype: int64
X: (6

无int操作 get_item_test: 12054    144
Name: item_id, dtype: int64
X: (71, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (71, 9)
无int操作 get_item_test: 12054    144
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 143
prediction:  [[3.43972859]]
无float操作 get_item_test: 12054    3
Name: rating, dtype: int64
77 over  943 users
无int操作 get_item_test: 3333    1160
Name: item_id, dtype: int64
X: (20, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (20, 9)
无int操作 get_item_test: 3333    1160
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 1159
prediction:  [[2.78142938]]
无float操作 get_item_test: 3333    5
Name: rating, dtype: int64
78 over  943 users
无int操作 get_item_test: 17278    1008
Name: item_id, dtype: int64
X: (54, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (54, 9)
无int操作 get_item_test: 17278    1008
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 1007
prediction:  [

无int操作 get_item_test: 19899    1238
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 1237
prediction:  [[3.00678048]]
无float操作 get_item_test: 19899    2
Name: rating, dtype: int64
100 over  943 users
无int操作 get_item_test: 92212    595
Name: item_id, dtype: int64
X: (66, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (66, 9)
无int操作 get_item_test: 92212    595
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 594
prediction:  [[2.76156138]]
无float操作 get_item_test: 92212    2
Name: rating, dtype: int64
101 over  943 users
无int操作 get_item_test: 39017    72
Name: item_id, dtype: int64
X: (215, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (215, 9)
无int操作 get_item_test: 39017    72
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 71
prediction:  [[2.6662626]]
无float操作 get_item_test: 39017    3
Name: rating, dtype: int64
102 over  943 users
无int操作 get_item_test: 1918    127
Name: item_id, dtype: int6

...

无int操作 get_item_test: 54297    748
Name: item_id, dtype: int64
X: (35, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (35, 9)
无int操作 get_item_test: 54297    748
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 747
prediction:  [[2.89387933]]
无float操作 get_item_test: 54297    4
Name: rating, dtype: int64
817 over  943 users
无int操作 get_item_test: 93971    288
Name: item_id, dtype: int64
X: (20, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (20, 9)
无int操作 get_item_test: 93971    288
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 287
prediction:  [[2.472122]]
无float操作 get_item_test: 93971    5
Name: rating, dtype: int64
818 over  943 users
无int操作 get_item_test: 79225    327
Name: item_id, dtype: int64
X: (27, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (27, 9)
无int操作 get_item_test: 79225    327
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 326
prediction:  [[4.

model.X[model.rated_items, :] (80, 9)
无int操作 get_item_test: 94333    241
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 240
prediction:  [[3.48565844]]
无float操作 get_item_test: 94333    4
Name: rating, dtype: int64
844 over  943 users
无int操作 get_item_test: 65605    1022
Name: item_id, dtype: int64
X: (30, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (30, 9)
无int操作 get_item_test: 65605    1022
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 1021
prediction:  [[2.87144583]]
无float操作 get_item_test: 65605    2
Name: rating, dtype: int64
845 over  943 users
无int操作 get_item_test: 65352    378
Name: item_id, dtype: int64
X: (404, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (404, 9)
无int操作 get_item_test: 65352    378
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 377
prediction:  [[3.79060414]]
无float操作 get_item_test: 65352    4
Name: rating, dtype: int64
846 over  943 users
无int操作 get_item_t

无int操作 get_item_test: 94386    909
Name: item_id, dtype: int64
X: (114, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (114, 9)
无int操作 get_item_test: 94386    909
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 908
prediction:  [[2.85088593]]
无float操作 get_item_test: 94386    3
Name: rating, dtype: int64
871 over  943 users
无int操作 get_item_test: 91241    290
Name: item_id, dtype: int64
X: (70, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (70, 9)
无int操作 get_item_test: 91241    290
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 289
prediction:  [[2.60547227]]
无float操作 get_item_test: 91241    2
Name: rating, dtype: int64
872 over  943 users
无int操作 get_item_test: 97673    348
Name: item_id, dtype: int64
X: (19, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (19, 9)
无int操作 get_item_test: 97673    348
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 347
prediction:  

无int操作 get_item_test: 93525    232
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 231
prediction:  [[4.44998926]]
无float操作 get_item_test: 93525    5
Name: rating, dtype: int64
897 over  943 users
无int操作 get_item_test: 80387    358
Name: item_id, dtype: int64
X: (29, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (29, 9)
无int操作 get_item_test: 80387    358
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 357
prediction:  [[3.81233468]]
无float操作 get_item_test: 80387    4
Name: rating, dtype: int64
898 over  943 users
无int操作 get_item_test: 74149    283
Name: item_id, dtype: int64
X: (134, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (134, 9)
无int操作 get_item_test: 74149    283
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 282
prediction:  [[3.24918926]]
无float操作 get_item_test: 74149    4
Name: rating, dtype: int64
899 over  943 users
无int操作 get_item_test: 82470    405
Name: item_id, dtype: i

无float操作 get_item_test: 70966    5
Name: rating, dtype: int64
923 over  943 users
无int操作 get_item_test: 75349    632
Name: item_id, dtype: int64
X: (81, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (81, 9)
无int操作 get_item_test: 75349    632
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 631
prediction:  [[3.82271728]]
无float操作 get_item_test: 75349    4
Name: rating, dtype: int64
924 over  943 users
无int操作 get_item_test: 86917    561
Name: item_id, dtype: int64
X: (31, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (31, 9)
无int操作 get_item_test: 86917    561
Name: item_id, dtype: int64
dataset.get_item_test(user) - 1: 560
prediction:  [[3.27708689]]
无float操作 get_item_test: 86917    3
Name: rating, dtype: int64
925 over  943 users
无int操作 get_item_test: 84277    258
Name: item_id, dtype: int64
X: (19, 9)
X_test: (1, 9)
model.X[test_items, :] (9,)
model.X[model.rated_items, :] (19, 9)
无int操作 get_item_test: 84277    25